<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo">
</center>


#### Import the required libraries we need for the lab.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

#### Read the dataset in the csv file from the URL


In [ ]:
boston_df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv')

#### Add your code below following the instructions given in the course to complete the peer graded assignment


## Task 1: Familiarize Yourself with the Dataset
Here we load and preview the Boston Housing dataset and review its key variables and statistics.

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

# Load dataset
boston_data = load_boston()
boston = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
boston['MEDV'] = boston_data.target
boston.columns = [col.lower() for col in boston.columns]

# Display first few rows
boston.head()


In [ ]:
# Summary statistics
boston.describe()


## Task 2: Generate Descriptive Statistics and Visualizations
This section includes boxplots, bar plots, scatter plots, and histograms with explanations.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Age groups
boston['age_group'] = pd.cut(boston['age'], bins=[0, 35, 70, 100], labels=['<=35', '36-70', '71+'])

# Create subplots
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
fig.suptitle('Descriptive Statistics and Visualizations', fontsize=18)

# Boxplot for MEDV
sns.boxplot(data=boston, y='medv', ax=axes[0, 0])
axes[0, 0].set_title("Boxplot of Median Home Value (MEDV)")
axes[0, 0].set_ylabel("Median Value ($1000s)")

# Bar plot for CHAS
chas_counts = boston['chas'].value_counts().sort_index()
axes[0, 1].bar(['Not Bound (0)', 'Bound (1)'], chas_counts, color=['skyblue', 'salmon'])
axes[0, 1].set_title("Bar Plot: Homes Bounded by Charles River")
axes[0, 1].set_ylabel("Count of Homes")

# Boxplot MEDV vs AGE group
sns.boxplot(data=boston, x='age_group', y='medv', ax=axes[1, 0])
axes[1, 0].set_title("Boxplot of MEDV by Age Group")
axes[1, 0].set_xlabel("Age Group")
axes[1, 0].set_ylabel("Median Value ($1000s)")

# Scatterplot of NOX vs INDUS
sns.scatterplot(data=boston, x='indus', y='nox', ax=axes[1, 1])
axes[1, 1].set_title("NOX vs INDUS")
axes[1, 1].set_xlabel("Proportion of Non-Retail Business Acres")
axes[1, 1].set_ylabel("NOX Concentration")

# Histogram of PTRATIO
sns.histplot(data=boston, x='ptratio', bins=15, kde=True, ax=axes[2, 0], color='orange')
axes[2, 0].set_title("Histogram of Pupil-Teacher Ratio")
axes[2, 0].set_xlabel("Pupil-Teacher Ratio")
axes[2, 0].set_ylabel("Frequency")

# Hide empty plot
axes[2, 1].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


## Task 3: Use the Appropriate Tests to Answer the Questions
We conduct hypothesis testing including t-test, ANOVA, Pearson correlation, and regression.

In [ ]:
from scipy.stats import ttest_ind, f_oneway, pearsonr
from sklearn.linear_model import LinearRegression

# T-test for CHAS
medv_river = boston[boston['chas'] == 1]['medv']
medv_no_river = boston[boston['chas'] == 0]['medv']
t_stat, p_val_ttest = ttest_ind(medv_river, medv_no_river, equal_var=False)

# ANOVA for AGE groups
group1 = boston[boston['age_group'] == '<=35']['medv']
group2 = boston[boston['age_group'] == '36-70']['medv']
group3 = boston[boston['age_group'] == '71+']['medv']
f_stat, p_val_anova = f_oneway(group1, group2, group3)

# Pearson correlation between NOX and INDUS
corr_coef, p_val_corr = pearsonr(boston['nox'], boston['indus'])

# Linear regression: DIS vs MEDV
X = boston[['dis']]
y = boston['medv']
reg_model = LinearRegression().fit(X, y)
slope = reg_model.coef_[0]
intercept = reg_model.intercept_
r_squared = reg_model.score(X, y)

# Display results
{
    "T-test (CHAS)": {"t_statistic": round(t_stat, 4), "p_value": round(p_val_ttest, 4)},
    "ANOVA (AGE groups)": {"f_statistic": round(f_stat, 4), "p_value": round(p_val_anova, 4)},
    "Pearson Correlation (NOX & INDUS)": {"correlation": round(corr_coef, 4), "p_value": round(p_val_corr, 4)},
    "Linear Regression (DIS → MEDV)": {"slope": round(slope, 4), "intercept": round(intercept, 4), "r_squared": round(r_squared, 4)}
}
